In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from IPython.display import Image, display
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

# Introduction

This notebook is my submission for the NFL Big Data Bowl 2022. 

The recently (2019) developed temporal fusion transformer (TFT) uses a multidimensional time series as input to forecast the multidimensional future. In this project, it predicts the next coordinates, speed, acceleration, direction, orientation, and distance of all 22 players in kickoff returns given past frames as input. By iteratively using the network's predictions as future input, one can continuously predict the next frame and simulate kickoff returns until the returner is tackled, runs out of bounds, or scores a touchdown.

Using a trained TFT, I ranked players based on hypothetical scenarios. A ranking system based solely on historical data is unnormalized in that teams and players face different teams and lineups in a given year increasing the factor of luck into rankings. My rankings are computed based on testing teams and players against the same set of conditions, i.e. the lineup of the most frequent player at each position, and could potentially provide a decomposition based on returners' out-of-system performances when returners are substituted into other teams' special teams lineups.

The tracking data is processed to a set of variables, some varying during each play (e.g. position) and some staying static, the same, (e.g. returner's weight), that represent a time instance. The aforementioned variables the network predicts are also considered as time varying input variables and are added to the input to the network when they precede the current time index. The static variables are as follows for each of the 22 players:

1. Weight
2. Height
3. Non-special teams position
4. 40 yd dash
5. Vertical jump
6. Bench press
7. Broad jump
8. 3Cone
9. Shuttle run
10. Draft pick number
11. Returner index
12. Kickoff team name
13. Returning team name
14. Play direction
15. Kicking team home or away.

Some additional pre-processing steps were applied. The variables were transformed so that the kickoff occurs on the right-hand side of the field to the left-hand side endzone. If combine data was not available for a player, the average score for that position was assigned to the player. Before all variables are input to the network, the players' names are sorted in increasing y-direction for the kicking team and increasing x-direction, then increasing y-direction for the receiving team. Data from 2018, 2019, and 2020 input into the trainer while the last 64 plays in 2020 were withheld, from which I conducted evaluations.

# Correlated Results Between Real and Simulated Return Yardage

In [ ]:
file = open("/kaggle/input/nfl2022-input-data/data/evaluations/yards_sim_vs_real.pkl",'rb')
sim_real_data = pd.DataFrame( pickle.load(file) )
file.close()

In [ ]:
sns.set_theme(color_codes=True)
slope, intercept, r_value, p_value, std_err = stats.linregress(sim_real_data['simulated'],sim_real_data['real'])

ax = sns.regplot(x="simulated", y="real", data=sim_real_data, line_kws={'label':"real={0:.2f}sim+{1:.2f}".format(slope,intercept)})

ax.legend()
plt.title('Simulated vs. Actual Return Yardage, r^2 = {0:.2f}, p = {1:.3f}'.format(r_value, p_value))
plt.show()

Figure 1 depicts a regression between the simulated and real return yardages for 64 plays which the neural network has not previously seen. There exists a positive, significant correlation of 0.38 between these values. There seems to exist a systematic bias, which can adjust for by using the net differences between simulated and actual to rank the players.

# Visual Comparison of Real and Simulated Kickoff Return

Below, are Figures 2 and 3 which a real play as well as the play simulated using just the first four frames. In the simulation, all players and the teams playing are different from that in the real. 

**Real Play**

In [ ]:
Image(url = "https://github.com/mattonics/NFL-BigData-Bowl-2022/blob/main/GIFs/anim_exam.gif?raw=true", width = 1000)

**Simulated Play**

In [ ]:
Image(url = "https://github.com/mattonics/NFL-BigData-Bowl-2022/blob/main/GIFs/anim_exam_sim.gif?raw=true", width = 1000)

# Performance of Rankings

Returners with their most frequent players at each blocking position were simulated against each team's kicking lineup with the most frequently appearing players at each position. 32 returners were simulated 32 times each on the same play starting from the frame the returner received the ball. Below, Figure 4 shows the ranking of how far the returner went on average when facing all 32 teams vs. the ranking of the returner's team average kickoff return yards in the 2020 season taken from NFL.com. The subplot on the right shows that although there does not exist a correlation on the whole, the data restricted to the top half of simulation yardage exhibits positive Spearman correlation of 0.45 with p-value of 0.08. The top ranked returner in simulation is Andre Roberts who was voted to the Pro Bowl in that season and belonged to the team with the highest average kickoff return yards in 2020, the Buffalo Bills.

In [ ]:
file = open("/kaggle/input/nfl2022-input-data/1-4-22 Upload/figure_data_ave_returns_simulated.pkl",'rb')
ave_returns_simulated = pickle.load(file)
file.close()

file = open("/kaggle/input/nfl2022-input-data/1-4-22 Upload/figure_data_ave_returns_team_ranked.pkl",'rb')
ave_returns_team_ranked = pickle.load(file)
file.close()

fig, axs = plt.subplots(1,2)
fig.set_size_inches(10.5, 5.75)
fig.tight_layout(pad=3.0)
fig.suptitle('Simulated vs. Historical Ave Return Yardage Ranking')
axs[1].scatter(ave_returns_simulated[ave_returns_simulated <= 16], ave_returns_team_ranked[ave_returns_simulated <= 16])
axs[1].set_aspect('equal')
plt.xlim(0,33)
plt.ylim(0,33)
axs[1].set_xlabel('Simulated Rank')
axs[1].set_ylabel('Actual Rank')
axs[1].set_title('Top Half of Simulated Rankings')
axs[0].scatter(ave_returns_simulated, ave_returns_team_ranked)
axs[0].set_aspect('equal')
plt.xlim(0,33)
plt.ylim(0,33)
axs[0].set_xlabel('Simulated Rank')
axs[0].set_ylabel('Actual Rank')
axs[0].set_title('All Players')
plt.show()


The full rankings of kickoff returners and their associated teams are shown below in Table 1.

In [ ]:
file = open("/kaggle/input/nfl2022-input-data/figures/player_system_rankings.txt",'rb')
rankings = file.read().decode('UTF-8')
file.close()
print(rankings)

# Explainability of AI

As the TFT neural network used for simulation utilizes an attention-based architecture it can be easily explainable. Below in Figure 5, we can see that only the preceding two frames of a single instant are much more useful compared to the previous eight to predict the next behavior of all the players on the field. A higher attention score means a greater weight is applied to that variable.

In [ ]:
Image("/kaggle/input/nfl2022-input-data/1-4-22 Upload 2/attention.png")

# Implications

1. As this prediction model does not require any historical data, NFL teams could use it to rank players from the NFL combine to assess how well they fit in with their set of blockers and their team prior to the season starting.
2. This model provides an assessment of how well the kick returners would perform outside of their team and could be used to scout good fits for trades or free agent signings.
3. This model could hopefully be extended to punt returns, rushing plays, passing plays, etc.

# Acknowledgements

I would like to thank Samira Kumar for the visualization tool to make animations of tracking data (https://github.com/samirak93/Game-Animation).